## Business Understanding

Choose stakeholder and figure out what they care about

- need two important parameter estimates as recommendations


## Exploratory Data Analysis

How/why does this data allow you to answer your questions?

**Data Visualization**
- Need 3 High Quality Data Visualizations

## Data Prep/Feature Eng

**Train/Test/Split**

**General Data Preparation**
- Nulls/dupes/outliers
- scaling
- transformations (for linearity)

**Feature Engineering**
- Encoding Variables (categories)
- Polynomial Features
- Interaction Terms
- NEED ONE NON-NUMERIC FEATURE

## Iterative Modeling

First try simplest approach.

Consider what the model baseline is

Start Adding Complexity and compare along the way
(**DOCUMENT THE ITERATIVE PROCESS**)

Models need to adhere to the assumptions of linear regression. If models do not adhere need to at least show that assumptions were tested for:

**ASSUMPTIONS**
- linear relationship between x and y - Heatmaps, scatter matrix
- low multicollinearituy - heatmaps, scatter matrix, VIF
- normal distribution of errors - kde, histogram, QQ-Plot, quantitative check (Jarque-Bera)
- homoskedasticity between errors - scatter plot between true and expected value

## Model Interpretation

Check for statistical Significance - check p-values of input variables

Are coefficients interpretable

Check R2, RMSE, MAE, MSE and other regression metrics

-----------------------


## STAKEHOLDER SELECTION

Stakeholder is Redfin

- Develop model to determine optimal purchasing price of homes and what renovations to make that will lead to greatest ROI


## BUSINESS UNDERSTANDING

**70% Rule:**

$$ 
MBP = (0.7ARV) - ERC
$$

where:

MBP = Maximum Buying Price

ARV = After-repair value

ERC = Estimated repair costs

in a buyers market we can adjust 70% to as high as 85%

## DATA EXPLORATION

In [178]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
import statsmodels
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor

In [179]:
num_features_to_select = 4
longitude_condition = False
latitude_condition = False

In [180]:
df = pd.read_csv('data/cleaned_data.csv')
df.drop(['id','date','Unnamed: 0','yr_renovated','sqft_living15'],axis=1,inplace=True)
df = df[df['yr_built'] > 1930]
df.reset_index(drop=True,inplace=True)
if latitude_condition:
    df = df[df['lat'] < 47.664]
if longitude_condition:
    df = df[df['long'] < -122.083]

In [85]:
# Canal Lat = 47.658491
# Canal Long = -122.3331

# City Lat = 47.664
# City Long = -122.083

In [205]:
from folium.plugins import FastMarkerCluster
import folium

lat = 47.4
long = -122.0

my_map = folium.Map([lat, long], zoom_start=10)

top_coordinates = [
    # Above Seattle 
    [47.777799, -122.419374],
    [47.664592, -122.419374],
    [47.664592, -121.966095],
    [47.777799, -121.966095],
    [47.777799, -122.419374],
] 
right_coordinates = [
    [47.664592, -121.966095],
    [47.510590, -121.966095],
    [47.510590, -122.151067]
]
coordinates = [
    # Seattle
    [47.473347, -122.151067],
    [47.664592, -122.151067],
    [47.664592, -122.419374],
    [47.473347, -122.419374],
    
    # Below Seattle
    [47.473347, -122.151067],
    [47.304134, -122.151067],
    [47.304134, -122.419374],
    [47.473347, -122.419374],
    [47.473347, -122.151067]
]
my_PolyLine = folium.PolyLine(locations = right_coordinates, weight = 5)
my_map.add_child(my_PolyLine)
my_PolyLine = folium.PolyLine(locations = top_coordinates, weight = 5)
my_map.add_child(my_PolyLine)
my_PolyLine=folium.PolyLine(locations=coordinates,weight=5)
my_map.add_child(my_PolyLine)
# add all the point from the file to the map object using FastMarkerCluster
my_map.add_child(FastMarkerCluster(df[['lat', 'long']].values.tolist()))

In [ ]:
# BOTTOM RIGHT
# Coordinates (47.469787, -122.177269) - (47.252278, -121.940139)

In [70]:
# sns.set(rc={'figure.figsize':(15, 15)})

# # Use the .heatmap method to depict the relationships visually!
# sns.heatmap(df.corr().abs(),annot=True);

In [71]:
# df_base = pd.read_csv('data/kc_house_data.csv')
# df_base.drop(['id','date','yr_renovated'],axis=1,inplace=True)
# df_base_corrs = df_base.corr()['price'].map(abs).sort_values(ascending=False)

In [72]:
# df_corrs = df.corr()['price'].map(abs).sort_values(ascending=False)

In [73]:
# df.filter(['grade','yr_built','lat','sqft_living'],axis=1).plot.kde(subplots=True,sharex=False)

In [74]:
# fig,axes = plt.subplots(nrows=2,ncols=2)

# df['grade'].hist(bins=9,ax=axes[0,0],legend=True)
# df['yr_built'].hist(bins=9,ax=axes[0,1],legend=True)
# df['sqft_living'].hist(bins=12,ax=axes[1,0],legend=True)
# df['lat'].hist(bins=10,ax=axes[1,1],legend=True)

In [ ]:
    [47.664592, -121.966095],
    [47.510590, -121.966095],
    [47.510590, -122.151067]

In [206]:
# df_seattle = df[(df['lat'] > 47.473347) & (df['lat'] < 47.664592)  & (df['long'] < -122.151067) & (df['long'] > -122.419374)]
# df_bottom = df[(df['lat'] > 47.304134) & (df['lat'] < 47.473347)  & (df['long'] < -122.151067) & (df['long'] > -122.419374)]
# df_top = df[(df['lat'] > 47.664592) & (df['lat'] < 47.777799)  & (df['long'] < -121.966095) & (df['long'] > -122.419374)]
# df_right = df[(df['lat'] > 47.510590) & (df['lat'] < 47.664592)  & (df['long'] < -121.966095) & (df['long'] > -122.151067)]
df_right

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,has_basement,yr_built,zipcode,lat,long,sqft_lot15,living_area
1,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,98074,47.6168,-122.045,7503,0.933333
5,400000.0,3,1.75,1370,9680,1.0,0,0,4,7,1370,0,1977,98074,47.6127,-122.045,10208,1.000000
14,550000.0,4,1.00,1660,34848,1.0,0,0,1,5,930,1,1933,98052,47.6621,-122.132,11467,0.768519
17,625000.0,4,2.50,2570,5520,2.0,0,0,3,9,2570,0,2000,98074,47.6145,-122.027,5669,1.040486
20,785000.0,4,2.50,2290,13416,2.0,0,0,4,9,2290,0,1981,98007,47.6194,-122.151,13685,0.854478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10619,425000.0,3,1.50,1390,9680,1.0,0,0,4,7,1390,0,1956,98008,47.6340,-122.125,10050,0.952055
10625,450000.0,2,1.00,1180,10720,1.0,0,0,4,7,1180,0,1955,98007,47.5893,-122.135,10750,0.830986
10648,541338.0,3,2.50,2060,8123,2.0,0,0,3,8,1010,1,1977,98052,47.6642,-122.130,6170,1.170455
10649,650000.0,4,3.00,2900,15535,1.0,0,1,4,7,1870,1,1961,98008,47.6120,-122.119,10217,1.244635


In [207]:
# Create Train and Test Datasets
X_train, X_test, y_train, y_test = train_test_split(df_top.drop('price', axis=1), df_top['price'], test_size=0.2, random_state=30)

In [208]:
# Fit Scalar to Train
ss = StandardScaler()
ss.fit(X_train)

# Transform both Train and Test
X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)

In [209]:
# Recursive Feature Selection
lr_rfe = LinearRegression()
select = RFE(lr_rfe, n_features_to_select=num_features_to_select)
select.fit(X=X_train_scaled, y=y_train)


keep_list = [(k,v) for k,v in zip(df.drop('price', axis=1).columns,select.support_)]
keep_string = ''
for k,v in keep_list:
    if v:
        keep_string += k + '+'
keep_string = keep_string[:-1]

In [210]:
# Creates scaled features dataframe and then adds on price column

# Train
X_train_scaled = pd.DataFrame(X_train_scaled)
X_train_scaled.columns = df.drop('price', axis=1).columns
y_train.reset_index(drop=True,inplace=True)
X_train_scaled_final = pd.concat((X_train_scaled,y_train),axis=1)

# Test
X_test_scaled = pd.DataFrame(X_test_scaled)
X_test_scaled.columns = df.drop('price', axis=1).columns
y_test.reset_index(drop=True,inplace=True)
X_test_scaled_final = pd.concat((X_test_scaled,y_test),axis=1)

In [211]:
# Baseline Model
baseline_mean = X_train_scaled_final['price'].mean()
model_base = DummyRegressor(strategy='mean', constant=baseline_mean)
baseline_mean

506824.1786447639

In [212]:
keep_string

'sqft_living+view+grade+lat'

In [213]:
#formula = f'price ~ {keep_string}'
formula = f'price ~ sqft_living+view+grade'
model = ols(formula,X_train_scaled_final).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.622
Model:                            OLS   Adj. R-squared:                  0.621
Method:                 Least Squares   F-statistic:                     1331.
Date:                Mon, 04 Oct 2021   Prob (F-statistic):               0.00
Time:                        17:20:03   Log-Likelihood:                -31559.
No. Observations:                2435   AIC:                         6.313e+04
Df Residuals:                    2431   BIC:                         6.315e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    5.068e+05   2087.551    242.784      0.000    5.03e+05    5.11e+05
sqft_living  7.616e+04   3162.590     24.082      0.000       7e+04    8.24e+04
view         2.663e+04   2137.103     12.461      0.000    2.24e+04    3.08e+04
grade        5.611e+04   3134.239     17.903      0.000       5e+04    6.23e+04
==============================================================================
Omnibus:                      453.108   Durbin-Watson:                   1.963
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1279.544
Skew:                           0.972   Prob(JB):                    1.41e-278
Kurtosis:                       5.972   Cond. No.                         2.69
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [214]:
model_test = ols(formula,X_test_scaled_final).fit()
model_test.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.608
Model:                            OLS   Adj. R-squared:                  0.606
Method:                 Least Squares   F-statistic:                     312.5
Date:                Mon, 04 Oct 2021   Prob (F-statistic):          1.74e-122
Time:                        17:20:04   Log-Likelihood:                -7867.7
No. Observations:                 609   AIC:                         1.574e+04
Df Residuals:                     605   BIC:                         1.576e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    5.032e+05   4021.649    125.134      0.000    4.95e+05    5.11e+05
sqft_living  7.111e+04   6259.096     11.361      0.000    5.88e+04    8.34e+04
view         1.914e+04   4291.542      4.460      0.000    1.07e+04    2.76e+04
grade        6.501e+04   6220.918     10.451      0.000    5.28e+04    7.72e+04
==============================================================================
Omnibus:                       48.222   Durbin-Watson:                   1.964
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               64.339
Skew:                           0.632   Prob(JB):                     1.07e-14
Kurtosis:                       3.969   Cond. No.                         2.57
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""